In [1]:
import numpy as np

In [ ]:
def compute_view_factor(N, H, S, A_j):
    """
    Compute the view factor between the fins or flat surface to the environment.
    
    Parameters:
    - N: Number of divisions (resolution) for integration
    - H: Height of the fins (L_fin) in meters
    - S: Spacing between the fins in meters
    - A_j: Area of the environment to which radiation is emitted (hemisphere)
    
    Returns:
    - View factor for fins and flat surface to the surroundings
    """
    # Create a grid of points for integration over the fin and flat surfaces
    z = np.linspace(0, H, N)  # Fin height divisions
    x = np.linspace(0, S, N)  # Fin spacing divisions
    dz = z[1] - z[0]  # Differential element for height
    dx = x[1] - x[0]  # Differential element for width (spacing)
    
    # Initialize view factor sums
    F_fin_sum = 0
    F_flat_sum = 0
    
    # Loop over all differential elements of the fin surface
    for z_i in z:
        for x_i in x:
            # Compute angles and distances
            s = np.sqrt(x_i**2 + z_i**2)  # Distance between element and environment
            if s == 0:
                continue  # Avoid division by zero
            
            cos_theta_i = z_i / s  # Cosine of angle for the fin
            cos_theta_j = 1  # For surrounding (environment assumed as hemisphere)

            # Differential view factor for the fins
            dF_fin = (cos_theta_i * cos_theta_j) / (np.pi * s**2) * A_j * dz * dx
            F_fin_sum += dF_fin
    
    # Loop over all differential elements of the flat surface
    for x_i in x:
        # Compute angles and distances
        s = x_i  # Distance for flat surface is horizontal spacing
        if s == 0:
            continue  # Avoid division by zero
        
        cos_theta_i = 1  # Flat surface faces upward, so cosine is 1
        cos_theta_j = 1  # For the surrounding hemisphere

        # Differential view factor for the flat surface
        dF_flat = (cos_theta_i * cos_theta_j) / (np.pi * s**2) * A_j * dx
        F_flat_sum += dF_flat
    
    return F_fin_sum, F_flat_sum

# Example usage:
H = 0.01  # Height of fins in meters
S = 0.005  # Spacing between fins in meters
A_j = 1.0  # Area of the environment (hemisphere above)
N = 100  # Number of divisions for integration

# Compute the view factors
F_fin_env, F_flat_env = compute_view_factor(N, H, S, A_j)

F_fin_env, F_flat_env

In [25]:
def compute_view_factor_corrected(H, S, depth, N) :
    """
    Compute the view factor between the fins or flat surface to the environment
    with proper normalization by the emitting surface area.
    
    Parameters:
    - N: Number of divisions (resolution) for integration
    - H: Height of the fins (L_fin) in meters
    - S: Spacing between the fins in meters
    - A_j: Area of the environment to which radiation is emitted (hemisphere)
    
    Returns:
    - View factor for fins computed using corrected approach.
    """
    # Create a grid of points for integration over the fin and flat surfaces
    z = np.linspace(0, H, N)  # Fin height divisions
    x = np.linspace(0, S, N)  # Fin spacing divisions
    dz = z[1] - z[0]  # Differential element for height
    dx = x[1] - x[0]  # Differential element for width (spacing)
    
    A_i = H * depth  # Total area of the fin surface
    
    # Initialize view factor sum
    F_fin_sum = 0
    
    # Loop over all differential elements of the fin surface
    for z_i in z:
        for x_i in x:
            # Compute angles and distances
            s = np.sqrt(x_i**2 + z_i**2)  # Distance between element and environment
            if s == 0:
                continue  # Avoid division by zero
            
            cos_theta_i = z_i / s  # Cosine of angle for the fin
            cos_theta_j = 1  # Environment is assumed to be hemispherical, so cos(theta_j) = 1

            # Differential view factor for the fins
            dF_fin = (cos_theta_i * cos_theta_j) / (np.pi * s**2) * (dz*depth) * (dx*depth)
            F_fin_sum += dF_fin

    # Normalize by the area of the fin surface (A_i)
    F_fin_env = F_fin_sum / A_i
    
    return F_fin_env

In [36]:
epsilon = 1e-12  # Small threshold to avoid singularity

In [ ]:
from scipy import integrate

def compute_view_factor_scipy(H, S, depth):
    """
    Compute the view factor between the fins or flat surface to the environment
    using robust integration via scipy's dblquad for double integration.
    
    Parameters:
    - H: Height of the fins (L_fin) in meters
    - S: Spacing between the fins in meters
    - depth: Depth of the fins (in the direction perpendicular to the height)
    
    Returns:
    - View factor for fins computed using scipy's integration approach.
    """
    
    # Define the integrand for the view factor
    def integrand(x_i, z_i):
        s = np.sqrt(x_i**2 + z_i**2 + epsilon)  # Distance between element and environment
        
        cos_theta_i = z_i / s  # Cosine of angle for the fin
        cos_theta_j = 1  # Assuming hemisphere, cos(theta_j) = 1 for the environment
        return (cos_theta_i * cos_theta_j) / (np.pi * s**2)
    
    # Set the limits of integration
    x_min, x_max = 0, S  # Fin spacing limits
    z_min, z_max = 0, H  # Fin height limits
    
    # Perform the double integration over the fin surface
    F_fin_sum, _ = integrate.dblquad(integrand, x_min, x_max, lambda x: z_min, lambda x: z_max)
    
    # Normalize by the area of the fin surface (A_i)
    A_i = H * depth  # Total area of the fin surface
    F_fin_env = F_fin_sum / A_i
    
    return F_fin_env

# Example usage:
H = 0.01  # Height of fins in meters
S = 0.005  # Spacing between fins in meters
depth = 1.  # Depth of the fins in meters

# Compute the view factor using scipy integration
F_fin_env_scipy = compute_view_factor_scipy(H, S, depth)

F_fin_env_scipy

In [ ]:
compute_view_factor_corrected(H, S, depth=1, N=100)  # Example usage

In [ ]:
compute_view_factor_corrected(H, S, depth=1, N=1000)  # Example usage

In [ ]:
S/(H+S)

In [ ]:
def view_factor(w_i, L):
    """
    Compute the view factor F_ij for parallel plates with w_j = w_i.
    
    Parameters:
    - w_i: Width of the plates (same for both)
    - L: Distance between the plates
    
    Returns:
    - F_ij: View factor between the two plates
    """
    W_i = w_i / L  # Normalize the width by the distance between plates
    F_ij = (np.sqrt(W_i**2 + 1) - 1) / W_i
    return F_ij

# Example usage
w_i = 0.020  # Width of the plates in meters
L = 0.020    # Distance between the plates in meters

F_ij = view_factor(w_i, L)
F_ij


In [ ]:
def view_factor_general(w_i, w_j, L):
    """
    Compute the view factor F_ij for parallel plates with different widths (w_i ≠ w_j).
    
    Parameters:
    - w_i: Width of plate i
    - w_j: Width of plate j
    - L: Distance between the plates
    
    Returns:
    - F_ij: View factor between the two plates
    """
    # Normalize the widths by the distance between the plates
    W_i = w_i / L
    W_j = w_j / L
    
    # Compute the view factor
    F_ij = ((np.sqrt((W_i + W_j)**2 + 4) - np.sqrt((W_j - W_i)**2 + 4)) / (2 * W_i))
    
    return F_ij

# Example usage
w_i = 100000 # Width of plate i in meters
w_j = 0.020  # Width of plate j in meters
L = 1.    # Distance between the plates in meters

F_ij_general = view_factor_general(w_i, w_j, L)
F_ij_general

In [ ]:


# Example usage:
r_i = 1.0  # Radius of disk i
r_j = 2.0  # Radius of disk j
L = 3.0    # Distance between the disks
view_factor_coaxial_disks(r_i, r_j, L)
